## Grabing the Data From the CitiBike S3 Bucket

In [14]:
import pandas as pd
import requests, glob, zipfile, io   # Needed to pull data from CitiBike S3 bucket


In [97]:
CITIBIKE_DATA_FOLDER = "https://s3.amazonaws.com/tripdata/"    

In [110]:
def pull_data(filename):
    try:
        r = requests.get(CITIBIKE_DATA_FOLDER + filename, stream=True)   
        r.raise_for_status()  
    except requests.exceptions.HTTPError as errh:
        return (errh)
    except requests.exceptions.ConnectionError as errc:
        return (errc)
    except requests.exceptions.Timeout as errt:
        return (errt)
    except requests.exceptions.RequestException as err:
        return (err)

    #.content is an attribute of the request.Response() object that returns the conent of object in bytes
    with zipfile.ZipFile(io.BytesIO(r.content), 'r') as data: 
        print(data.namelist())

In [114]:
pull_data("202006-citibike-tripdata.csv.zip")

['202006-citibike-tripdata.csv', '__MACOSX/', '__MACOSX/._202006-citibike-tripdata.csv']


In [67]:
stream_str = io.BytesIO(b"JournalDev Python: \x00\x01")
print(b"JournalDev Python: \x00\x01")
print(stream_str.getvalue())


b'JournalDev Python: \x00\x01'
b'JournalDev Python: \x00\x01'
